In [1]:
import re
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

splits = {'train': 'sent_train.csv', 'validation': 'sent_valid.csv'}
df = pd.read_csv("hf://datasets/zeroshot/twitter-financial-news-sentiment/" + splits["train"])

In [2]:
df

,text,label
0,$BYND - JPMorgan reels in expectations on Beyo...,0
1,$CCL $RCL - Nomura points to bookings weakness...,0
2,"$CX - Cemex cut at Credit Suisse, J.P. Morgan ...",0
3,$ESS: BTIG Research cuts to Neutral https://t....,0
4,$FNKO - Funko slides after Piper Jaffray PT cu...,0
...,...,...
9538,The Week's Gainers and Losers on the Stoxx Eur...,2
9539,Tupperware Brands among consumer gainers; Unil...,2
9540,vTv Therapeutics leads healthcare gainers; Myo...,2
9541,"WORK, XPO, PYX and AMKR among after hour movers",2


In [3]:
TICKER_PATTERN = re.compile(r'\$[A-Z]+')
URL_PATTERN = re.compile(r'(https:\/\/[^\s]+)')

def row_parser(row: str) -> dict:
    tickers = re.findall(TICKER_PATTERN, row)
    url = re.findall(URL_PATTERN, row)
    return {'url': '' if len(url) == 0 else url[0], 'tickers': tickers}

parsed_list = []
for row in df.text:
    parsed_list.append(row_parser(row))
parsed_df = pd.DataFrame(parsed_list)

In [4]:
df = df.join(parsed_df)
df

,text,label,url,tickers
0,$BYND - JPMorgan reels in expectations on Beyo...,0,https://t.co/bd0xbFGjkT,[$BYND]
1,$CCL $RCL - Nomura points to bookings weakness...,0,https://t.co/yGjpT2ReD3,"[$CCL, $RCL]"
2,"$CX - Cemex cut at Credit Suisse, J.P. Morgan ...",0,https://t.co/KN1g4AWFIb,[$CX]
3,$ESS: BTIG Research cuts to Neutral https://t....,0,https://t.co/MCyfTsXc2N,[$ESS]
4,$FNKO - Funko slides after Piper Jaffray PT cu...,0,https://t.co/z37IJmCQzB,[$FNKO]
...,...,...,...,...
9538,The Week's Gainers and Losers on the Stoxx Eur...,2,https://t.co/iK5MO4NhQq,[]
9539,Tupperware Brands among consumer gainers; Unil...,2,,[]
9540,vTv Therapeutics leads healthcare gainers; Myo...,2,,[]
9541,"WORK, XPO, PYX and AMKR among after hour movers",2,,[]


In [9]:
options = Options()
options.add_argument('--headless=new')
driver = webdriver.Chrome(options=options)
body_text_list = []
l = len(df['url'])
print(f'Total urls to process: {l}')
for i in range(l):
    url = df['url'][i]
    if len(url):
        driver.get(url)
        soup = BeautifulSoup(driver.page_source)
        body_text_list.append({"article": soup.text})
    if i%50 == 0:
        print(f'{i}/{l}')
    else:
        body_text_list.append({"article": ""})   
driver.close()
print("Done!")

Total urls to process: 9543
0/9543
50/9543
100/9543
150/9543
200/9543
250/9543
300/9543
350/9543
400/9543
450/9543
500/9543
550/9543
600/9543
650/9543
700/9543
750/9543
800/9543
850/9543
900/9543
950/9543
1000/9543
1050/9543
1100/9543
1150/9543
1200/9543
1250/9543
1300/9543
1350/9543
1400/9543
1450/9543
1500/9543
1550/9543
1600/9543


KeyboardInterrupt: 